In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import gzip

In [5]:
import numpy as np

def load_emnist_data(data_path, labels_path):
    with open(data_path, 'rb') as f:
        # Skip the first 16 bytes for images file (magic number + dimensions)
        data = np.frombuffer(f.read(), dtype=np.uint8, offset=16)
    with open(labels_path, 'rb') as f:
        # Skip the first 8 bytes for labels file (magic number + item count)
        labels = np.frombuffer(f.read(), dtype=np.uint8, offset=8)
    return data.reshape(-1, 28, 28), labels


In [8]:
train_images_path = "D:\\github\\text-scanner\\ver.1\\balanced_extracted\\emnist-balanced-train-images-idx3-ubyte"
train_labels_path = "D:\\github\\text-scanner\\ver.1\\balanced_extracted\\emnist-balanced-train-labels-idx1-ubyte"
test_images_path = "D:\\github\\text-scanner\\ver.1\\balanced_extracted\\emnist-balanced-test-images-idx3-ubyte"
test_labels_path = "D:\\github\\text-scanner\\ver.1\\balanced_extracted\\emnist-balanced-test-labels-idx1-ubyte"


In [9]:
# Load the training and test data
x_train, y_train = load_emnist_data(train_images_path, train_labels_path)
x_test, y_test = load_emnist_data(test_images_path, test_labels_path)

In [10]:
# Normalize the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [11]:
# Reshape to add channel dimension for CNN
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [12]:
# One-hot encode the labels
num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [13]:
print(f"Training data shape: {x_train.shape}, Training labels shape: {y_train.shape}")
print(f"Test data shape: {x_test.shape}, Test labels shape: {y_test.shape}")


Training data shape: (112800, 28, 28, 1), Training labels shape: (112800, 47)
Test data shape: (18800, 28, 28, 1), Test labels shape: (18800, 47)


In [14]:
# Build the CNN model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

C:\Users\thecr\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

Epoch 1/10
705/705 ━━━━━━━━━━━━━━━━━━━━ 15s 20ms/step - accuracy: 0.5928 - loss: 1.4655 - val_accuracy: 0.8251 - val_loss: 0.5245
Epoch 2/10
705/705 ━━━━━━━━━━━━━━━━━━━━ 16s 23ms/step - accuracy: 0.8389 - loss: 0.4857 - val_accuracy: 0.8484 - val_loss: 0.4361
Epoch 3/10
705/705 ━━━━━━━━━━━━━━━━━━━━ 15s 21ms/step - accuracy: 0.8627 - loss: 0.3967 - val_accuracy: 0.8576 - val_loss: 0.4045
Epoch 4/10
705/705 ━━━━━━━━━━━━━━━━━━━━ 15s 21ms/step - accuracy: 0.8743 - loss: 0.3539 - val_accuracy: 0.8661 - val_loss: 0.3819
Epoch 5/10
705/705 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.8868 - loss: 0.3176 - val_accuracy: 0.8641 - val_loss: 0.3862
Epoch 6/10
705/705 ━━━━━━━━━━━━━━━━━━━━ 15s 21ms/step - accuracy: 0.8927 - loss: 0.2894 - val_accuracy: 0.8661 - val_loss: 0.3716
Epoch 7/10
705/705 ━━━━━━━━━━━━━━━━━━━━ 15s 21ms/step - accuracy: 0.8997 - loss: 0.2651 - val_accuracy: 0.8671 - val_loss: 0.3811
Epoch 8/10
705/705 ━━━━━━━━━━━━━━━━━━━━ 15s 21ms/step - accuracy: 0.9065 - loss: 0.2482 - 

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")

In [ ]:
# Save the model for future use
model.save('emnist_character_recognition.h5')